# EE4211 Project

## Question 1

Group name:

Group members: Liu Tianshu, Nie Hongtuo, Pan Jie, Zhang Chenxi

### Question 1.1

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('dataport-export_gas_oct2015-mar2016.csv')

In [ ]:
# turn to timestamp
def local2utc(time):
    return pd.Timestamp(time).tz_convert("UTC")
df.localminute = df.apply(lambda r: local2utc(r.localminute), axis=1)

First let's get the number of houses.

In [ ]:
grouped = df.groupby(['dataid'])
len(grouped)

or

In [ ]:
df.dataid.value_counts().size

We have gas flow meters installed in 157 houses.
Now we can find the anomaly samples in the data.

In [ ]:
defect_id = []
for key,group in grouped:
    val = group.meter_value.values
    diff = val[1:] - val[:-1]
    defect_time = np.where(diff < 0)[0]
    if len(defect_time):
        defect_id.append(key)
        defect_time = group.localminute.values[defect_time + 1]
        print('The glitches of gas meter No.', key, 'happened in the moments below:')
        print(defect_time, '\n')

In [ ]:
print('The ID of the', len(defect_id), 'defective meters are:')
print(defect_id)

43 gas meters have sent values that decreased at some point.
Let's look inside the data series given by a defective meter, for example, gas meter No. 9982.

In [ ]:
group = grouped.get_group(9982)
t = group.localminute.values
val = group.meter_value.values

s = pd.Series(val, index=t)
s.plot(figsize=(16, 8))
plt.xticks(rotation=-30)

### Question 1.2
Generate hourly readings from the raw data. Select one month from the 6-month
study interval and plot the hourly readings (time-series) for that month. Hint: You
will have to decide what to do if there are no readings for a certain hour.

In [ ]:
import datetime
def local2utc(time):
    return pd.Timestamp(time).tz_convert("UTC")
def select_data(df, month) -> pd.DataFrame:
    column_names = df.columns.values
    values = df.values
    values = filter(lambda x: x[0].month == month, values)
    values = pd.DataFrame(values)
    for i in range(len(column_names)):
        values = values.rename(columns={i:column_names[i]})
    return values

In [ ]:
## select Oct data
month_data = select_data(df, 10)
print(month_data)

In [ ]:
def hourly_data_by_group(df) -> set():
    groups = df.groupby(['dataid'], sort=['localminute'])
    processed_group = {}
    for key in groups.groups.keys():
        new_group = []
        current_group = groups.get_group(key).values
        # init_first_point
        first_point = current_group[0]
        first_point[0] = first_point[0].replace(day = 1, hour = 0, minute = 0, second = 0)
        index_current_group = 0
        tmp = []
        # replace all time on the hour
        for row in current_group:
            row[0] = row[0].replace(minute = 0, second = 0)
            tmp.append(row)
        current_group = tmp
        # interpolate data 24 hour for 30 day
        for day in range(1, 31):
            for hour in range(0, 24):
                # make sure its the last data of the current time
                while (True):
                    if (index_current_group >= len(current_group)):
                        break
                    current_row = current_group[index_current_group]
                    time = current_row[0]
                    if (time.day == day and time.hour == hour):
                        index_current_group += 1
                    else:
                        break
                # assign data
                current_row = current_group[index_current_group - 1]
                time = current_row[0]
                date = pd.Timestamp(year = time.year, month = time.month, day = time.day, hour = time.hour, minute = 0, second = 0)
                current_row = [date, current_row[1], current_row[2]]
                if (time.day == day and time.hour == hour):
                    if (len(new_group) > 0 and new_group[-1][0].day == day and new_group[-1][0].hour == hour):
                        new_group[-1] = current_row
                    else:
                        new_group.append(current_row)
                else:
                    date = pd.Timestamp(year = time.year, month = time.month, day = day, hour = hour, minute = 0, second = 0)
                    row = [date,new_group[-1][1],new_group[-1][2]]
                    new_group.append(row)
        processed_group[key] = new_group
    return processed_group

In [ ]:
hourly_data_groups = hourly_data_by_group(month_data)

In [ ]:
print(hourly_data_groups.keys(), '\n')
for i in hourly_data_groups[35]:
    print(i)